In [19]:
from N3_FL import *
from json import loads
from U2_Merkle import *

In [20]:
def truncateRepus(repus):
    nrepus = []
    for r in repus:
        nrepus.append({'hash': r['hash'][:5], 'qual_mined' : "{:.2f}".format(r['qual_mined']), 'qual_reported' : "{:.2f}".format(r['qual_reported'])})
    return nrepus

In [21]:
# Add bogus payment model
class Transaction:
    def __init__ (self, chain, sender, recipient, load, type_, payment): #type_ - mod/pay/glb
        self.sender = sender
        self.load = load
        self.recipient = recipient
        self.type = type_
        self.payment = payment
        self.chain = chain
    
    def __str__(self):
        r = "TRX: C{0:5.5}:{1:5.5}-->{2:5.5} | {3:5.5} | ¤{4:4.2f} | Load: ".format(str(self.chain), str(self.sender), str(self.recipient), self.type, self.payment) + self.load
        return r

In [22]:
class Block:
    def __init__(self, chain, chain_idx, trxs, proof, prev_hash, ts=None, repu_list=None):
        self.chain = chain
        self.chain_idx = chain_idx
        self.transactions = trxs
        self.proof = proof
        self.prev_hash = prev_hash
        if ts == None:
            self.timestamp = time()
        else:
            self.timestamp = ts
        if repu_list == None:
            self.reputation_list = []
        else:
            self.reputation_list = repu_list
        self.merkle = buildTree([toHash(x) for x in trxs])
        self.tree_hash = objPickleToIPFS(IPFS(0), self.merkle, "./temp")

    def __str__(self):
        r = "BLK: C:{0:7.7}#{1:3d} <-- PH={2:5.5}:PF={3:5.5}, RL={4:5.5}, #TRXS={5}".format(str(self.chain.bc_id),self.chain_idx,str(self.prev_hash),str(self.proof),str(self.reputation_list), str(len(self.transactions)))
        return r

In [23]:
def removeCli(txns, cli): #in dict
    tx_new = []
    for t in txns:
        if t.type == 'PAY':
            tx_new.append(t)
        if t.type == 'MOD':
            if t.sender == cli:
                pass
            else:
                tx_new.append(t)
    return tx_new

def truncateViewRL(rl):
    r = ""
    for c, q in rl.items():
        r += "{0:5.5}:{1:.4}; ".format(c,q)
    return r

def remDuplicate(lst):
    lst_nw = []
    check = []
    for i in lst:
        if i['hash'] not in check:
            check.append(i['hash'])
            lst_nw.append(i)
    return lst_nw

In [24]:
# MAKE CONVERTER TO OTHER ROLE VIA TRX DATA?????
class Node(Client):
    def __init__(self, id_):
        super().__init__(id_)
        self.blockchain = None
        self.balance = 0
        self.committee = True
        self.mine_benchmarks = []
        self.current_trxs = []

    def initSet(self):
        if hasattr(self, 'blockchain') == False:
            self.blockchain = None
            self.balance = 0
    
    def __str__(self):
        return "{0}:{3}-{1:5.5}, {2:7d}".format(self.getType(), \
                            self.hash, self.getVolume(), self.id_)

    def getType(self):
        return "NDE"

    def assignBlockchain(self, blockchain):
        self.blockchain = blockchain
        self.blockchain.authorizeClient(self)
                        
    def volAsHash(self):
        loc_vol = './temp/t1.txt'
        vol_hash = self.ipfs.sendToIPFS(saveToFile(str(self.getVolume()/self.roundLimit), loc_vol))
        rm(loc_vol)
        return vol_hash

    def submitModelAsTransaction(self):
        # send model HASH and MULTIPLIER (number of data)
        load = "{}|{}|{}|{}".format(self.hash, self.hashes[0], self.hashes[1], self.volAsHash()) # mod, qual, vol
        trx = Transaction(self.blockchain.bc_id, self.hash, 0, load, 'MOD', 0)
        chidx = self.blockchain.newTransaction(trx)
        print("{0}:{1} MOD-->CHIDX#{2:3d}".format(self.getType(),str(self.id_), chidx))

    def evaluateMine(self, model_weights):
        model_ = CNN()
        model_.setup()
        model_.replaceModel(model_weights)
        qual = model_.model.evaluate(self.test_inp, self.test_oup, verbose=2)
        return qual[1]
    
    def cliTrainPack(self, epoch_, r):
        loc = './models/'
        self.train(loc, epoch_, r)
        self.submitModelAsTransaction()

# COMMITTEE FUNCTIONS_________________________________________________________________________________
    def aggregate(self, blockchain):
        if self.committee == False:
            return
    
        # Initiate resulting global model
        global_model = None
        
        loc = './models/'
        self.mine_benchmarks = []
        # IMPORT PAST QUALITIES
        # REPU LIST: QUAL (VOL * ACC)
        past_qualities = blockchain.lastBlock.reputation_list

        for t in blockchain.current_transactions:
            if t.type == 'MOD':
                # parse t
                cli_hash, mod_hash, qual_hash, vol_hash = t.load.split('|')
                model_ = IPFStoModObj(self.ipfs, mod_hash, loc)
                vol_ = int(float(IPFStoObj(self.ipfs, vol_hash, loc)))
                qual_ = float(IPFStoObj(self.ipfs, qual_hash, loc))
                
                # Test everything against itself
                # qual_ = REPORTED qual = vol ** accu
                # compare to c.evalmine
                evalmine = self.evaluateMine(model_)
                benchmarks = {'hash': cli_hash, 'qual_mined' : evalmine * (vol_ ** 0.5), \
                                'vol' : vol_, 'qual_reported' : qual_, 'model_file' : model_.get_weights()}
                if evalmine > 0.5:
                    self.mine_benchmarks.append(benchmarks)

        # Quality check from previous versions of qualities
        # calculate past qualities average first

        # get sum of past qualities
        sum_qual = 0
        for record in past_qualities:
            sum_qual += record['qual_mined']

        past_average_qual = 0
        for record in past_qualities:
            past_average_qual += record['qual_mined'] / sum_qual

        improvement_threshold = 0.9
        average_threshold = 0.3
        bm_pass = []
        for bm in self.mine_benchmarks:
            if len(past_qualities) > 0:
                for record in past_qualities: # THIS ONE WONT RUN IF EMPTY
                    if bm['hash'] in record['hash']:
                        # LOC_NOW vs LOC_PAST
                        if float(bm['qual_mined']) > improvement_threshold * float(record['qual_mined']):
                            print(float(bm['qual_mined']), 'old:', float(improvement_threshold * record['qual_mined']), 'LOC NOW vs LOC PAST')
                            bm_pass.append(bm)
                            continue
                        # LOC_NOW vs GLOB_PAST
                        if float(bm['qual_mined']) > average_threshold * past_average_qual:
                            print(float(bm['qual_mined']), 'old:', average_threshold * past_average_qual, 'LOC NOW vs GLOB PAST')
                            bm_pass.append(bm)
                    else:
                        bm_pass.append(bm)
            else:
                bm_pass.append(bm)
        self.mine_benchmarks = remDuplicate(bm_pass)

        sel_hashes = []
        for h in self.mine_benchmarks:
            sel_hashes.append(h['hash'])

        # Aggregate from benchmarks
        # formula: qual_report + qual_mined
        if len(self.mine_benchmarks) > 1:
            models_multiply = []
            models_multiply_proportion = []
            models = []    
            for bm in self.mine_benchmarks:
                # get weights and models
                models_multiply.append(bm['qual_mined'] + bm['qual_reported'])
                models.append(bm['model_file'])
            
            for m in models_multiply:
                models_multiply_proportion.append(m / sum(models_multiply))
            
            average = combine_ultimate(models, models_multiply_proportion)

            # aggregate
            global_model = CNN()
            global_model.setup()
            global_model.model.set_weights(average)
            global_model.setData(self.train_inp, self.train_oup)
            global_model.train('', sess)
        elif len(self.mine_benchmarks) == 1:
            global_model = CNN()
            global_model.setup()
            global_model.model.set_weights(self.mine_benchmarks[0]['model_file'])
            global_model.setData(self.train_inp, self.train_oup)
            global_model.train('', sess)
        else:
            #NO MINE
            print('Nothing to mine')

        
        print('GLOB EVAL ', self.evaluateMine(global_model.model))

        # REMOVE TRX FROM THOSE NOT IN FINAL C MINE BENCHMARKS (FILTER)
        trx_new = []
        for t in self.blockchain.current_transactions:
            if t.sender in sel_hashes or t.type == 'GLO':
                trx_new.append(t)

        hash_weight_global = objPickleToIPFS(self.ipfs, global_model.model.get_weights(), './temp/')
        mine_benchmarks = objPickleToIPFS(self.ipfs, truncateRepus(self.mine_benchmarks), './temp/')
        
        trx_global = Transaction(self.blockchain, self.hash, 0, "{}|{}|{}".format(self.hash, hash_weight_global, mine_benchmarks), 'GLO', 0)
        
        self.current_trxs = trx_new      
        self.current_trxs.append(trx_global)
        self.blockchain.newTransaction(trx_global)


In [25]:
class Compounder(Node):
    def __init__(self, id_):
        super().__init__(id_)
        self.ocNodes = []
        self.model_eval = CNN()
        self.model_eval.setup()
    
    def initSet(self):
        if hasattr(self, 'ocNodes') == False:
            self.ocNodes = []
            self.model_eval = CNN()
            self.model_eval.setup()

    def getType(self):
        return "CPD"

    def registerOCNode(self, node):
        self.ocNodes.append(node)

    def getVolume(self):
        vol_ocs = 0
        for o in self.ocNodes:
            vol_ocs += o.getVolume()
        return len(self.train_inp) + vol_ocs

    def train(self, loc, epoch_, round_):
        self.model.model.set_weights(self.combineModels())
        return super().train(loc, epoch_, round_)

    # NEED TO HAVE SOME OFFCHAIN MODIF
    def replaceWeights(self, weights):
        self.model.model.set_weights(weights)
        #self.model.model.build((None,self,36,1))
        self.model.model.build((None,self,28,28,1))
        self.model.model.compile(optimizer='adam',loss='mse', metrics=['accuracy'])
        for oc in self.ocNodes:
            oc.replaceWeights(weights)

    def cliTrainPack(self, epoch_, r):
        loc = './models/'
        for oc in self.ocNodes:
            oc.train(loc, epoch_, r)
        self.train(loc, epoch_, r)
        self.submitModelAsTransaction()

    def examine(self):
        m_evals = []
        m_lens = []
        m_mods = []
        for oc in self.ocNodes:
            self.model_eval.replaceModel(oc.model.model)
            mo = oc.model.model.get_weights()
            le = oc.getVolume()
            ev = self.model_eval.model.evaluate(self.test_inp, self.test_oup, verbose=2)
            m_evals.append(ev)
            m_lens.append(le)
            m_mods.append(mo)
        m_evals.append(self.model.model.evaluate(self.test_inp, self.test_oup, verbose=2))
        m_lens.append(self.getVolume())
        m_mods.append(self.model.model.get_weights())
        return m_mods, m_lens, m_evals

    def combineModels(self):
        c_mods, c_lens, c_evals = self.examine()
        c_contribs = []
        c_props = []
        
        for ci in range(len(c_evals)):
            c_contribs.append(c_evals[ci][1] * (c_lens[ci] ** 0.5))

        for ci in range(len(c_evals)):
            c_props.append(c_contribs[ci]/sum(c_contribs))
        
        mod_combined = combine_ultimate(c_mods, c_props)
        return mod_combined

In [26]:
class OffChainNode(Client):
    def __init__(self, id_):
        super().__init__(id_)
        #self.id_ = toHash(id_)

    def initSet(self):
        pass
    
    def getType(self):
        return "OCN"

    def __str__(self):
        return "{0}:{3}-{1:5.5}, {2:7d}".format(self.getType(), \
                            self.hash, self.getVolume(), self.id_)
    
    def requestBind(self, cli):
        cli.registerOCNode(self)

In [27]:
def saveModelH(model, ipfs):
    save_loc = './models/glob0'
    model.model.save(save_loc, save_format='h5')
    hash = ipfs.sendToIPFS(save_loc)
    rm(save_loc)
    return hash

def saveFileToLoc(content, loc):
    f = open(loc, 'w')
    f.write(content)
    f.close()
    return loc

In [28]:
class Blockchain(object): # PROVING NODE: ID + SALT + HASH
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.bc_id = toHash(id(self))
        self.clients = set()
        self.cli_pointers = []
        self.newBlock(Block(self,0, [Transaction(self, 0, 0, "Genesis", "GEN", 0)], 0, 0, []))
        self.salt = str(randint(1,100000)) # salt for this blockchain
        print("Blockchain created with ID: " + self.bc_id[:4])
        self.ipfs = IPFS(self.bc_id)

    def authorizeClient(self, client):
        # hash is secret to the
        cid_hash = toHash(client.id_ + self.salt)
        self.clients.add(cid_hash)
        self.cli_pointers.append(client)

    def proofOfAuthority(self, clientId):
        if toHash(clientId + self.salt) not in self.clients:
            print("{0:5.5} MINE REJECTED: UNAUTH".format(clientId))
            return False
        print("{0:5.5} MINE APPROVED: POA".format(clientId))
        return True

    def newTransaction(self, trx):
        self.current_transactions.append(trx)
        return self.lastBlock.chain_idx + 1

    @property
    def lastBlock(self):
        return self.chain[-1]

    def newBlock(self, block):
        self.chain.append(block)
        self.current_transactions = []
        return block


In [29]:
def truncateListTime(lst, decimal):
    s = ''
    for l in lst:
        s += '{:.2f} | '.format(l)
    return s
def printRepus(repus):
    for r in repus:
        print(r['hash'][:5], r['qual_mined'])

In [30]:
# for c in clients: train, send model to IPFS - get hash, send hash to blockchain as trx
# for m in miners: get trxs from blockchain backlog, 
def clientBatchRun(blockchain, cli, epoch_, round_, cli_poi=None):
    # Train, send model to IPFS, get hash
    for rnd in range(round_):
        print('ROUND {} =====================================--=-=-=-==-=-=-=-=--=-=-=-=-=-=========================='.format(rnd))
        # Train off-chain first

        print('TRAINING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Train and submit ===============================================
        t_training = []
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                continue
            print(ci.hash[:5], 'in operation: TRAINING.')
            ###################### MOVE TO BC
            if ci.blockchain.proofOfAuthority(ci.id_) == False:
                print("Unauthorized")
                continue
            t00 = time()
            ci.cliTrainPack(epoch_, rnd)
            t01 = time()
            t_training.append(t01 - t00)

        print('AGGREGATION>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        t_mine = []
        # Miners gather data ==============================================
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                print('Class not Node')
                continue
            t10 = time()
            print(ci.hash[:5], 'in operation: MINE.')
            ci.aggregate(ci.blockchain)
            t11 = time()
            t_mine.append(t11 - t10)

        # REPU HERE FROM GLOBAL MODELS
        # CONSENSUS PART 2

        print('CONSENSUS>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        cli_new_inc = []
        t_consensus = []
        threshold_miner = 0.5
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                print('Class not Node')
                continue
            ev = ci.evaluate()[1]
            if ev < 0.5:
                continue
            t20 = time()
            print("Client", ci.hash[:5])
            new_increments = []
            increment = 0
            improving_clis = set()
            old_repus = truncateRepus(ci.blockchain.lastBlock.reputation_list)
            if len(old_repus) == 0:
                for t in blockchain.current_transactions:
                    if t.type == 'GLO' and t.sender == ci.hash:
                        c_hash, c_weight_glob_hash, c_repu_hash = t.load.split('|') # change repu hash to one's evaluation
                        c_repus = IPFStoPickleObj(IPFS(0), c_repu_hash, './temp/')
                        for r in c_repus:
                            print('CREPUS: -evaluate', r['qual_mined'], ev)
                            increment += float(r['qual_mined']) * ev
                            improving_clis.add(r['hash'])
            else:
                for i in old_repus:
                    print('RL =', i['hash'][:5], i['qual_mined'])
                for i in c_repus:
                    print('RM =', i['hash'][:5], i['qual_mined'])
                for t in blockchain.current_transactions:
                    if t.type == 'GLO':
                        c_hash, c_weight_glob_hash, c_repu_hash = t.load.split('|')
                        c_repus = IPFStoPickleObj(IPFS(0), c_repu_hash, './temp/')
                        print("CRPEUS: ",c_repus)

                        # compare with RL
                        tolerance = 0.95
                        for i in range(len(c_repus)): # WRONG LOGIC
                            for r in range(len(old_repus)):
                                if c_repus[i]['hash'] == old_repus[r]['hash']:
                                    old_rep = float(old_repus[r]['qual_mined'])
                                    new_rep = float(c_repus[i]['qual_mined'])
                                    print('old repu for', c_repus[i]['hash'], ':', old_rep, 'new:', new_rep, 'inc:', new_rep - old_rep)
                                    incr = (new_rep - tolerance * old_rep) * ev
                                    if incr > 0:
                                        increment += incr
                                        c_repus[i] = old_repus[r]
                                        improving_clis.add(t.sender)

            new_increments.append({'miner_hash':ci.hash, 'increment':increment * log(ci.getVolume(),100), 'improving':improving_clis})
            print("NINCRE: ", end='')
            for ni in new_increments:
                print("NINCRE: ", ni['increment'])
            cli_new_inc.append(new_increments[0])
            t21 = time()
            t_consensus.append(t21 - t20)

        print('WINNING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        #select max increment -> move to blockchain === CONSENSUS
        best = 0
        winning_miner = None
        print('CLI-NEW-INC: ', cli_new_inc)
        for n in range(len(cli_new_inc)):
            #print(cli_new_inc[n]['miner_hash'], best, '>', cli_new_inc[n]['increment'])
            if cli_new_inc[n]['increment'] > best:
                best = cli_new_inc[n]['increment'] 
                winning_miner = cli_new_inc[n]['miner_hash']
                #print('winner changed to', cli_new_inc[n]['miner_hash'])
        try:
            print('Winner: ', winning_miner[:5])
            # exclude others' GLO
            for tx in blockchain.current_transactions:
                if tx.type == 'GLO' and tx.sender != winning_miner:
                    blockchain.current_transactions.remove(tx)
        except:
            print("No winners. Round skipped.")
            continue

        print('BLOCK PROPOSAL>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Client makes a block + merkleize, propose it to the blockchain
        proposed_block = None
        for ci in cli:
            if ci.hash == winning_miner:
                print(ci.hash[:5], 'in operation: BLOCK PUSH.')
                proposed_block = Block(ci.blockchain, len(ci.blockchain.chain), ci.current_trxs, ci.hash, toHash(ci.blockchain.lastBlock), time(), ci.mine_benchmarks)
                ci.blockchain.newBlock(proposed_block) #IMPLEMENT
                print('Block created: ', toHash(ci.blockchain.lastBlock), ' with #trxs ', len(ci.blockchain.lastBlock.transactions), ' and miner ', ci.blockchain.lastBlock.proof[:5])
                print('Blockchain length now = ', len(ci.blockchain.chain))
                print("NREPU: ", truncateRepus(ci.mine_benchmarks))
                print('TRXS:')
                for t in ci.blockchain.lastBlock.transactions:
                    print('tx', t.sender, t.type)

        print('GLOBAL IMPORT>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Clients import global model from winning miner
        t_import = []
        for ci in cli:
            t30 = time()
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                print('Class not Node')
                continue
            # import winning 
            for tx in ci.blockchain.lastBlock.transactions:
                if tx.type == 'GLO' and tx.sender == winning_miner:
                    weights = IPFStoPickleObj(ci.ipfs, tx.load.split('|')[1], './temp/')
                    ci.replaceWeights(weights) # Compounder needs imple here
            t31 = time()
            t_import.append(t31 - t30)

        cli[0].blockchain.current_transactions = []

        print('QUALITY CONTROL>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')        
        # CHECK QUALITY
        fileopen = open('./temp/session-training-bfl.txt', 'a')
        fileopen.write('SESS {} || ep={} |it={}/{} poi={}\n'.format(sess, epoch_, rnd + 1, round_, cli_poi))
        fileopen.write('Training  times: ' + truncateListTime(t_training, 2) + '\n')
        fileopen.write('Mining    times: ' + truncateListTime(t_mine, 2) + '\n')
        fileopen.write('Consensus times: ' + truncateListTime(t_consensus, 2) + '\n')
        fileopen.write('Import    times: ' + truncateListTime(t_import, 2) + '\n')
        fileopen.write('Winner={}\n'.format(winning_miner[:3]))
        fileopen.write('Evaluation: \n')
        for ci in cli:
            evalua = ci.evaluate()[1]
            print('EVAL', ci.id_, evalua)
            fileopen.write('{} | {:.4f}; '.format(ci.hash[:3], evalua))
        fileopen.write('\nReputation: \n')
        fileopen.write(str(truncateRepus(blockchain.lastBlock.reputation_list)))
        fileopen.write('\n---------------------------------------------------------------\n')
        fileopen.close()

        # ↑↑↑↑↑ SEND TO TXT FOR EASY READING
        # ADD ANNOTATIONS FOR POISONING

        f = open('./temp/bfl-bench.txt')
        # if time for committee change:
            # do committee change
        print("FINAL REPUS: ")
        printRepus(blockchain.lastBlock.reputation_list)
        print('round {} done\n\n'.format(rnd))

In [31]:
def getSessionBFL():
    f = open('./temp/bfl-bench.txt', 'r')
    ses = int(f.read())
    f.close()
    f = open('./temp/bfl-bench.txt', 'w')
    f.write(str(ses + 1))
    f.close()
    return str(ses + 1)

In [32]:
model_dir = "./models/"
count = 7
test_ratio = 0.25
sess = getSessionBFL()

try:
    train_inps = pickle.load(open("./temp/trinps", "rb"))
    train_oups = pickle.load(open("./temp/troups", "rb"))
    test_inps = pickle.load(open("./temp/teinps", "rb"))
    test_oups = pickle.load(open("./temp/teoups", "rb"))
    test_inp = pickle.load(open("./temp/teinp", "rb"))
    test_oup = pickle.load(open("./temp/teoup", "rb"))
except:
    train_inp, test_inp, train_oup, test_oup = split(fetch(), test_ratio)
    train_inps = buildPortions(count, train_inp)
    train_oups = buildPortions(count, train_oup)
    test_inps = buildPortions(count, test_inp)
    test_oups = buildPortions(count, test_oup)
    pickle.dump(train_inps, open("./temp/trinps", "wb"))
    pickle.dump(train_oups, open("./temp/troups", "wb"))
    pickle.dump(test_inps, open("./temp/teinps", "wb"))
    pickle.dump(test_oups, open("./temp/teoups", "wb"))
    pickle.dump(test_inp, open("./temp/teinp", "wb"))
    pickle.dump(test_oup, open("./temp/teoup", "wb"))

classes_initial = [Node, Node, Node, Compounder, Node, OffChainNode, OffChainNode]

In [33]:
def cliFL():
    epochs = 2
    rounds = 2

    # At first, everyone is node | Assumed held global model
    clients = setupClisP(train_inps, train_oups, test_inps, test_oups, rounds, [], classes_initial)
    binds = [[6,3],[5,3]]
    bindOCNodes(binds, clients)

    blockchain_ = Blockchain()
    # Assign blockchain
    for cli in clients:
        if cli.__class__ != OffChainNode:
            cli.assignBlockchain(blockchain_)

    clientBatchRun(blockchain_, clients, epochs, rounds)

In [34]:
def setupClisP(train_inps_, train_oups_, test_inps_, test_oups_, rounds_, client_poi, classes_initial):
    clients = []
    for i in range(len(train_inps_)):
        c = classes_initial[i](i)
        c.model.setup()
        if int(c.id_) in client_poi:
            print('poisoned: ', c.id_)
            c.setData(poisonBFLin(train_inps_[i]), poisonBFLout(train_oups_[i]), test_inps_[i], test_oups_[i])
        else:
            c.setData(train_inps_[i], train_oups_[i], test_inps_[i], test_oups_[i])
        c.setRoundLimit(rounds_)
        clients.append(c)
    return clients
    
def bindOCNodes(binds, clients):
    # binds: [cliOC, cliComp]
    for b in binds:
        clients[b[0]].requestBind(clients[b[1]])

In [35]:
client_poisoning = [[0],[],[5,4],[6,4,3],[1],[4,2,1],[2,0]]
ep_it = [(2,2),(4,1),(8,1),(16,1),(1,2),(2,1),(4,2),(8,2),(1,4),(2,4),(4,4),(2,8)]

cpo = [[1],[4,2,1],[2,0]]
ei = [(16,1)]
def clientBatchRunPoi(cli_poi_, ep_it_):
    for ep, it in ep_it_:
        for i in range(len(cli_poi_)):
            print("EP = {:2d} IT = {:2d}, POI = ".format(ep, it) + str(cli_poi_[i]))
            clis_poi = setupClisP(train_inps, train_oups, test_inps, test_oups, it, cli_poi_[i], classes_initial)
            binds_poi = [[6,3],[5,3]]
            bindOCNodes(binds_poi, clis_poi)

            # Start here
            blockchain_ = Blockchain()
            # Assign blockchain
            for cli in clis_poi:
                if cli.__class__ != OffChainNode:
                    cli.assignBlockchain(blockchain_)

            clientBatchRun(blockchain_, clis_poi, ep, it, cli_poi_[i])

ADD RULE: IF MINE < ...% AND VOLUME > ... THEN REJECT = MARK AS MALICIOUS
AFT THAT EXCLUDE FROM 

REPUS NOT UPDATING

In [36]:
#clientBatchRunPoi(client_poisoning, ep_it)
clientBatchRunPoi(cpo, ei)
#cliFL()

EP = 16 IT =  1, POI = [0]
poisoned:  0
Blockchain created with ID: eff4
ROUND 0 =====================================--=-=-=-==-=-=-=-=--=-=-=-=-=-==========================
TRAINING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
dfd5f in operation: TRAINING.
0     MINE APPROVED: POA
Cli# 0: 74.87 sec, Q: 0.0614; NDE:0 MOD-->CHIDX#  1
e2538 in operation: TRAINING.
1     MINE APPROVED: POA
Cli# 1: 43.04 sec, Q: 0.8661; NDE:1 MOD-->CHIDX#  1
58b2a in operation: TRAINING.
2     MINE APPROVED: POA
Cli# 2: 23.60 sec, Q: 0.8066; NDE:2 MOD-->CHIDX#  1
4cfc3 in operation: TRAINING.
3     MINE APPROVED: POA
Cli# 6: 2.84 sec, Q: 0.7264; Cli# 5: 5.00 sec, Q: 0.7603; 35/35 - 0s - loss: 0.0592 - accuracy: 0.6760
35/35 - 1s - loss: 0.0527 - accuracy: 0.7260
35/35 - 0s - loss: 0.1146 - accuracy: 0.0227
Cli# 3: 16.76 sec, Q: 0.7935; CPD:3 MOD-->CHIDX#  1
271f9 in operation: TRAINING.
4     MINE APPROVED: POA
Cli# 4: 7.89 sec, Q: 0.8131; NDE:4 MOD-->CHIDX#